In [1]:
#import ipywidgets
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, Dropdown, IntSlider
from IPython.display import clear_output

In [2]:
df = pd.read_csv('../datasets/tidy_data.csv')

In [3]:
#SET DEFAULT VALUES
season =  20162017
seasonW = Dropdown(options = df['season'].unique(), value = season)

game_type = 'regular_season'
game_typeW = Dropdown(options=['playoffs','regular_season'], value = game_type)
 
games = df[(df['season']==season) & (df['type_season']==game_type)]['id_game'].unique()
gameW = IntSlider(value = 0,min=0,max=len(games)-1,continuous_update=False)


events = df[(df['season']==season) & (df['type_season']==game_type) & (df['id_game']==games[0])]
eventW = IntSlider(value = 0,min=0,max=len(events)-1,continuous_update=False)

#Plot Debugger
@interact(season = seasonW,game_type = game_typeW,game = gameW,event=eventW)
def debugger(season,game_type,game,event):
    
    games = df[(df['season']==season) & (df['type_season']==game_type)]['id_game'].unique()
    gameW.max = len(games) - 1
    
    if game > len(games):
        game = 0
        gameW.value = 0
    
    events = df[(df['season']==season) & (df['type_season']==game_type) & (df['id_game']==games[game])]
    eventW.max =  len(events) - 1
    
    if event >  len(events):
        event = 0
        eventW.value = 0
    
    event_infos = events.iloc[event]
    
    fig, ax = plt.subplots()
    img = plt.imread("../figures/nhl_rink.png")
    ax.imshow(img, extent=[-100, 100, -43, 43])
    
    season_title = f'{str(season)[:4]}-{str(season)[4:]}'
    home = event_infos['team_home_name']
    away = event_infos['team_away_name']
    plt.suptitle(f'{season_title}\n\n{home} VS {away}')
    
    
    shot_type =  event_infos['shot_type']
    event_type =  event_infos['result_event']
    player =  event_infos['shooter_name']
    
   
     
    plt.title(f'A {shot_type} of {player}: {event_type}')
   
   
    
    x =  event_infos['x_coord']
    y =  event_infos['y_coord']
    ax.scatter(x, y, marker='o', color='r', label='point')
    
    clear_output(wait=True)

interactive(children=(Dropdown(description='season', options=(20162017, 20172018, 20182019, 20192020, 20202021…